In [1]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierV2
from textgrad.loss import TextLoss

### Setup LLM Engine

In [2]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setup Solution to Optimize

In [3]:
initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")


### Set Loss Variable

In [4]:
loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                              requires_grad=False,
                              role_description="system prompt")

### Inisiate Optimizer

In [5]:
optimizer = TextualGradientDescent([solution])

### Find Loss

In [6]:
loss = TextLoss(loss_system_prompt, engine=engine)
loss_value = loss(solution) # Forward method in Loss Function

print("INITIAL LOSS:", loss_value)

INITIAL LOSS: The calculation of  b² - 4ac was incorrect: it should be (-7)² - 4 * 3 * 2 = 49 - 24 = 25, not 49 + 24 = 73.  Also, the division by 6 is missing in the final answers for x1 and x2.



### Using TextualVerifier to Verify Loss

In [7]:
# Verify Loss
verifier = TextualVerifierV2(
    verifier_engine=engine, 
    verification_iterations=3, 
    verification_focus="Numerical accuracy and loss computation correctness",
    logger=True)

verification_prompt = Variable("""Evaluate loss value for correctness and reasonable magnitude.""",
                              requires_grad=False,
                              role_description="system prompt")

verified_result = verifier.verify(instance=solution, 
                                    prompt=verification_prompt,
                                    calculation=loss_value)

print("VERIFIED RESULT:", verified_result.value)

INFO: Starting generic textual verification...
INFO: Analyzing calculation...
INFO: Generating improved variants...
INFO: Selecting best variant...
INFO: Making final decision...
DECISION: Using improved version
INFO: Generic verification complete!
VERIFIED RESULT: To solve the equation 3x² - 7x + 2 = 0, we use the quadratic formula:

x = (-b ± √(b² - 4ac)) / 2a

Here, a = 3, b = -7, and c = 2.

Substituting these values into the quadratic formula:

x = ( -(-7) ± √((-7)² - 4 * 3 * 2) ) / (2 * 3)

x = ( 7 ± √(49 - 24) ) / 6

x = ( 7 ± √25 ) / 6

x = ( 7 ± 5 ) / 6

This gives us two solutions:

x₁ = (7 + 5) / 6 = 12 / 6 = 2

x₂ = (7 - 5) / 6 = 2 / 6 = 1/3

Therefore, the solutions to the equation 3x² - 7x + 2 = 0 are x = 2 and x = 1/3.


While a loss function isn't typically used when solving a quadratic equation (as the goal is to find the exact solutions), we can demonstrate a loss calculation for illustrative purposes.  Let's consider the Mean Squared Error (MSE) as an example.  If we

### Optimize Solution with Optimizer (TGD)

In [8]:
# TRANSFER: Replace the loss value with verified result
loss_value.set_value(verified_result.value) 

print("FINAL LOSS (after transfer):", loss_value.value)

FINAL LOSS (after transfer): To solve the equation 3x² - 7x + 2 = 0, we use the quadratic formula:

x = (-b ± √(b² - 4ac)) / 2a

Here, a = 3, b = -7, and c = 2.

Substituting these values into the quadratic formula:

x = ( -(-7) ± √((-7)² - 4 * 3 * 2) ) / (2 * 3)

x = ( 7 ± √(49 - 24) ) / 6

x = ( 7 ± √25 ) / 6

x = ( 7 ± 5 ) / 6

This gives us two solutions:

x₁ = (7 + 5) / 6 = 12 / 6 = 2

x₂ = (7 - 5) / 6 = 2 / 6 = 1/3

Therefore, the solutions to the equation 3x² - 7x + 2 = 0 are x = 2 and x = 1/3.


While a loss function isn't typically used when solving a quadratic equation (as the goal is to find the exact solutions), we can demonstrate a loss calculation for illustrative purposes.  Let's consider the Mean Squared Error (MSE) as an example.  If we had target values for x, we could compare our calculated solutions to these targets.  For this demonstration, let's *arbitrarily* assume the target values were x_target₁ = 2 and x_target₂ = 0.33 (an approximation of 1/3).

The MSE is ca

In [9]:
# Optimize
loss_value.backward()

optimizer.step()
print("SOLUTION:", solution.value)

["Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \nDo not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:\nx = (-b ± √(b^2 - 4ac)) / 2a\na = 3, b = -7, c = 2\nx = (7 ± √((-7)^2 + 4(3)(2))) / 6\nx = (7 ± √73) / 6\nThe solutions are:\nx1 = (7 + √73)\nx2 = (7 - √73) </LM_INPUT>\n\n<LM_OUTPUT> To solve the equation 3x² - 7x + 2 = 0, we use the quadratic formula:\n\nx = (-b ± √(b² - 4ac)) / 2a\n\nHere, a = 3, b = -7, and c = 2.\n\nSubstituting these values into the quadratic formula:\n\nx = ( -(-7) ± √((-7)² - 4 * 3 * 2) ) / (2 * 3)\n\nx = ( 7 ± √(49 - 24) ) / 6\n\nx = ( 7 ± √25 ) / 6\n\nx = ( 7 ± 5 ) / 6\n\nThis gives us two solutions:\n\nx₁ = (7 + 5) / 6 = 12 / 6 = 2\n\nx₂ = (7 - 5) / 6 = 2 / 6 = 1/3\n\nTherefore, the solutions to the equation 3x² - 7x + 2 = 0 are x = 2 and x = 1